### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности.

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [1]:
!pip install razdel
!pip install datasets
!pip install transformers
!pip install seqeval
!pip install accelerate>=0.20.1 -U
# !pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=60ae0197be817c1a2aa5729cf64ce865e2b2f09cd2f95e5fb1053b15c63557d3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
import random
import torch
import numpy as np
random_seed = 39
random.seed(random_seed)
np.random.seed(random_seed)
torch.random.manual_seed(random_seed)

In [3]:
# считаем данные
import pandas as pd
data = pd.read_csv("/kaggle/input/ner-data-train/ner_data_train.csv")

In [4]:
data.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"{""label"":""локация""\,""offset"":26\,""length"":6\,""..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"{""label"":""организация""\,""offset"":196\,""length""..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"{""label"":""название проекта""\,""offset"":12\,""len..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"{""label"":""не найдено""\,""offset"":162\,""length"":..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","{""label"":""команда""\,""offset"":13\,""length"":8\,""..."


In [5]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать,
# удалить лишние '\' например, преобразовать из str в список dict-ов
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

In [6]:
df.head(3)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"[{'label': 'локация', 'offset': 26, 'length': ..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"[{'label': 'организация', 'offset': 196, 'leng..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"[{'label': 'название проекта', 'offset': 12, '..."


#### Оригинал туториала на медицинских данных можно посмотреть тут https://gist.github.com/avidale/cacf235aebeaaf4c578389e1146c3c57

B - начало, I - продолжение

In [7]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах

    raw_toks = list(tokenize(item['video_info']))
    # print(raw_toks)
    words = [tok.text for tok in raw_toks]
    # print(words)
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        # print(char2word[word.start:word.stop], [i] * len(word.text))
        char2word[word.start:word.stop] = [i] * len(word.text)
    # char2word - на разных словах разные номера "При как" - [0,0, None, 1, 1]
    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    # print(char2word)
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено': ### - такое возможно?
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                # print(char2word[e['offset']:e['offset']+e['length']], e_words, e['label'])
                # e_words - номера различных слов "При как" - [0,0, None, 1, 1] - {0, 1}
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

extract_labels(df.iloc[0])

{'tokens': ['<',
  'НАЗВАНИЕ',
  ':',
  '>',
  'Агент',
  '117',
  ':',
  'Из',
  'Африки',
  'с',
  'любовью',
  '—',
  'Русский',
  'тизер',
  '=',
  'трейлер',
  '(',
  '2021',
  ')',
  '<',
  'ОПИСАНИЕ',
  ':',
  '>',
  'Лучший',
  'Telegram',
  'канал',
  'о',
  'кино',
  '<',
  'LINK',
  '>',
  'Сотрудничество',
  '<',
  'LINK',
  '>',
  'Дата',
  'выхода',
  '26',
  'августа',
  '2021',
  'Оригинальное',
  'название',
  ':',
  'OSS',
  '117',
  ':',
  'Alerte',
  'rouge',
  'en',
  'Afrique',
  'noire',
  'Страна',
  ':',
  'Франция',
  'Режиссер',
  ':',
  'Николя',
  'Бедос',
  'Жанр',
  ':',
  'боевик',
  ',',
  'комедия',
  'В',
  'главных',
  'ролях',
  ':',
  'Жан',
  'Дюжарден',
  ',',
  'Пьер',
  'Нинэ',
  ',',
  'Мелоди',
  'Каста',
  ',',
  'Наташа',
  'Линдинжер',
  ',',
  'Владимир',
  'Иорданов',
  ',',
  'Фату',
  'Н',
  '’',
  'Диайе',
  ',',
  'Пол',
  'Уайт',
  'Мир',
  'изменился',
  '.',
  'Он',
  'нет',
  '.',
  'Судьба',
  'заносит',
  'легендарного',
  'Аге

### Обработаем датасет и разобьем на трейн и тест

In [8]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
test_size = 0.15
val_size = 0.15
ner_train, ner_val = train_test_split(ner_data, test_size=test_size + val_size, random_state=random_seed)
ner_val, ner_test = train_test_split(ner_val, test_size=test_size/(test_size + val_size), random_state=random_seed)
assert len(ner_train) + len(ner_val) + len(ner_test) == len(data)

In [10]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
1330,"[<, НАЗВАНИЕ, :, >, Картинг, ., Лемар, ., Соревнования, на, приз, картодрома, Лемар, на, прокатных, картах, ., Квалификация, ., mp, 4, <, ОПИСАНИЕ, :, >, Интересный, технический, спорт, ., Занятия, по, картингу, в, закрытом, помещении, в, детской, школе, ., В, видео, показывается, ,, как, происх...","[O, O, O, O, O, O, B-локация, O, O, O, O, O, B-локация, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-название проекта, O, O, O, O, O, O, O, B-название проекта, O, O, O, O, O, O, O, B-название про..."
3170,"[<, НАЗВАНИЕ, :, >, Ремонт, и, обслуживание, кондиционеров, у, метро, Новокузнецкая, !, <, ОПИСАНИЕ, :, >, Ремонт, и, обслуживание, сплит, систем, у, метро, Новокузнецкая, !, Мы, делаем, различные, услуги, :, Ремонт, кондиционеров, ,, чистка, ,, заправка, ,, обслуживание, ,, установка, и, многое...","[O, O, O, O, O, O, O, O, O, B-локация, I-локация, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4435,"[<, НАЗВАНИЕ, :, >, Немецкая, усадьба, в, Калининградской, области, ., Довоенная, архитектура, ., Осмотрели, вместе, с, Заброшки, 39, <, ОПИСАНИЕ, :, >, Заброшки, 39, путешествуй, с, нами, !, Найдем, попутчиков, ,, познакомим, с, единомышленниками, ,, расскажем, интересные, факты, об, историческ...","[O, O, O, O, O, O, B-локация, I-локация, I-локация, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-локация, I-локация, I-локация, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-организация, O, O, O, O, O, O, B-организация, O, O, O, O, O, O..."


#### Посмотрим на получившиеся теги
Подробнее почитать про BIO теги можно тут https://datascience.stackexchange.com/questions/63399/what-is-bio-tags-for-creating-custom-ner-named-entity-recognization

In [11]:
def check_labels():
    res = []
    ma = 0
    for i in [ner_train, ner_test, ner_val]:
        label_list = sorted({label for item in ner_train for label in item['tags']})
        if 'O' in label_list:
            label_list.remove('O')
            label_list = ['O'] + label_list
        res.append(label_list)
    if not (len(res[0]) == len(res[1]) and len(res[0]) == len(res[2])):
        raise "Different size"
    for i in range(len(res[0])):
        if not (res[0][i] == res[1][i] and res[0][i] == res[2][i]):
            print(res[0][i], res[1][i], res[2][i])
            raise "Different values"
check_labels()

In [12]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list, len(label_list)

(['O',
  'B-Дата',
  'B-бренд',
  'B-вид спорта',
  'B-видеоигра',
  'B-команда',
  'B-лига',
  'B-локация',
  'B-модель',
  'B-название проекта',
  'B-организация',
  'B-персона',
  'B-сезон',
  'B-серия',
  'I-Дата',
  'I-бренд',
  'I-вид спорта',
  'I-видеоигра',
  'I-команда',
  'I-лига',
  'I-локация',
  'I-модель',
  'I-название проекта',
  'I-организация',
  'I-персона',
  'I-сезон',
  'I-серия'],
 27)

In [13]:
from datasets import Dataset, DatasetDict

In [14]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'val':  Dataset.from_pandas(pd.DataFrame(ner_val)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5715
    })
    val: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 642
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 65
    })
})

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [15]:
def get_num_params(model):
    res = 0
    for i in model.parameters():
        x = 1
        for j in i.shape:
            x *= j
        res += x
#     print(res)
    return res

In [16]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# model_checkpoint = "cointegrated/rubert-tiny"

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='cpu')
# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
# get_num_params(model)

In [17]:
model_checkpoint = "surdan/LaBSE_ner_nerel"#"cointegrated/rubert-tiny"#"xlm-roberta-large-finetuned-conll03-english"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='cpu')
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=27, ignore_mismatched_sizes=True)
get_num_params(model)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at surdan/LaBSE_ner_nerel and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([58, 768]) in the checkpoint and torch.Size([27, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([58]) in the checkpoint and torch.Size([27]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


127775259

In [18]:
for i in model.parameters():
    print(i.shape)

torch.Size([55083, 768])
torch.Size([512, 768])
torch.Size([2, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size

In [19]:
example = ner_train[5]
print(example["tokens"])

['<', 'НАЗВАНИЕ', ':', '>', 'ЗАКУПКА', 'ПРОДУКТОВ', 'НА', 'ОГРОМНОМ', 'УЗБЕКСКОМ', 'БАЗАРЕ', 'ЧОРСУ', '/', 'в', 'Корее', 'этого', 'нет', '!', 'Виктория', 'Ким', '<', 'ОПИСАНИЕ', ':', '>', 'Вот', 'и', 'долгожданная', 'закупка', 'продуктов', 'с', 'базара', 'Чорсу', '!', 'Покажу', 'вам', 'какими', 'продуктами', 'мы', 'обычно', 'закупаемся', 'в', 'Ташкенте', 'и', 'сам', 'базар', '.', 'Пишите', 'свои', 'комментарии', 'и', 'ставьте', 'лайки', '!', 'канал', 'Ники', ':', '<', 'LINK', '>', '/', '<', 'AT', '>', '/', 'videos', 'Ники', 'в', 'un', 'с', 't', ':', 'nikymacaleen', 'канал', 'Коли', ':', '<', 'LINK', '>', 'телеграм', '<', 'LINK', '>', 'вк', '<', 'LINK', '>', 'мы', 'в', 'un', 'с', 't', ':', 'Вика', '<', 'AT', '>', 'Коля', '<', 'AT', '>', '00', ':', '00', 'Вступление', '01', ':', '25', 'Гуляем', 'по', 'базару', '06', ':', '17', 'Ищем', 'виноград', '07', ':', '18', 'Смотрим', 'арбузы', '08', ':', '09', 'Покупаем', 'лепёшки', '09', ':', '11', 'Купили', 'виноград', '10', ':', '24', 'Показыва

In [20]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '<', 'НА', '##З', '##ВА', '##НИ', '##Е', ':', '>', 'ЗА', '##К', '##УП', '##КА', 'ПРО', '##ДУ', '##К', '##ТО', '##В', 'НА', 'О', '##Г', '##РО', '##М', '##НО', '##М', 'У', '##З', '##БЕ', '##КС', '##КО', '##М', 'БА', '##ЗА', '##Р', '##Е', 'Ч', '##ОР', '##С', '##У', '/', 'в', 'Кор', '##ее', 'этого', 'нет', '!', 'Виктория', 'Ким', '<', 'ОП', '##ИС', '##АН', '##ИЕ', ':', '>', 'Вот', 'и', 'долго', '##жда', '##нная', 'за', '##купка', 'продуктов', 'с', 'базар', '##а', 'Ч', '##орс', '##у', '!', 'Пока', '##жу', 'вам', 'какими', 'продуктами', 'мы', 'обычно', 'за', '##купа', '##емся', 'в', 'Ташкент', '##е', 'и', 'сам', 'базар', '.', 'П', '##ишите', 'свои', 'комментарии', 'и', 'став', '##ьте', 'лайк', '##и', '!', 'канал', 'Ники', ':', '<', 'LINK', '>', '/', '<', 'AT', '>', '/', 'videos', 'Ники', 'в', 'un', 'с', 't', ':', 'ni', '##ky', '##ma', '##cale', '##en', 'канал', 'Коли', ':', '<', 'LINK', '>', 'теле', '##грам', '<', 'LINK', '>', 'в', '##к', '<', 'LINK', '>', 'мы', 'в', 'un', 'с', 't'

In [21]:
len(example["tags"]), len(tokenized_input["input_ids"])

(169, 255)

In [22]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

255 255


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [23]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [24]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [25]:
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [26]:
# Специальный объект для удобного формирования батчей
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [27]:
metric = load_metric("seqeval")

In [28]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'вид спорта': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'команда': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 5},
 'лига': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'название проекта': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'организация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'персона': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 6},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [29]:
# Посчитаем метрики на отложенном датасете
def get_test_metrics(verbose = False, x = None):
    if x is None:
        predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
    else:
        predictions, labels, _ = trainer.predict(x)

    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if verbose:
        return results
    else:
        return {'overall_precision' : results["overall_precision"], 'overall_recall' : results['overall_recall'],
                'overall_f1' : results['overall_f1'], 'overall_accuracy' : results['overall_accuracy']}
    return results

In [30]:
batch_size = 8
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [31]:
# что мы видим без дообучения модели
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 3.539114236831665,
 'eval_precision': 0.00742519719737364,
 'eval_recall': 0.09451689805076427,
 'eval_f1': 0.013768730273842478,
 'eval_accuracy': 0.018975242723017958,
 'eval_runtime': 12.8405,
 'eval_samples_per_second': 49.998,
 'eval_steps_per_second': 6.308}

In [32]:
get_test_metrics()

{'overall_precision': 0.00869061413673233,
 'overall_recall': 0.10344827586206896,
 'overall_f1': 0.016034206306787813,
 'overall_accuracy': 0.022388059701492536}

In [33]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [34]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [35]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, patience = 3, threshold = 1e-3)
optimizers = (optimizer, scheduler)

In [36]:
batch_size = 32
args = TrainingArguments(
    output_dir = "train_checkpoints",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    num_train_epochs=65,

    save_total_limit = 5,
    weight_decay=0.001,
    save_strategy="epoch",
    report_to='none',
    metric_for_best_model = "f1",
    load_best_model_at_end = True,
    label_smoothing_factor = 0.001,
    auto_find_batch_size = True,
    use_cpu = False
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers = optimizers
)

In [37]:
args.per_device_eval_batch_size

8

In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.389100,0.344655,0.537640,0.430655,0.478237,0.896667
2,0.338500,0.346604,0.499369,0.554901,0.525673,0.904568


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

In [39]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

### Посмотрим на результаты

In [ ]:
# text = ' '.join(ner_train[25]['tokens'])
text = ner_train[25]['tokens']

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cpu')

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}

    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:
            print(f'{t:15s} {label_list[idx]:10s}')
    return text, pipe(text), labels

In [ ]:
predict_ner(text, tokenizer, model, pipe)

### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [ ]:
from tqdm.notebook import tqdm

submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')
def sample_submission(text, tokenizer, model, pipe, submission):
    for i, elem in tqdm(enumerate(ner_test)):
        _, _, labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
        submission.loc[i, 'video_info'] = elem

        submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
    return submission

In [ ]:
result = sample_submission(text, tokenizer, model, pipe, submission)

In [ ]:
result

In [ ]:
len(ner_test)